### Importing necessary libraries

In [6]:
import logging
import sys
from typing import Optional
import os.path
import openai
### remove credential while running this is used to hide API-KEY
from credential import API_KEY
### put your api-key here
openai.api_key = API_KEY

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    ServiceContext,
    set_global_service_context,
 
    
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import ServiceContext, set_global_service_context,get_response_synthesizer
from llmsherpa.readers import LayoutPDFReader
from llama_index.core.schema import Document


### Generate two llm models one for answering and other for follow-up question
#### Follow-up question needs creativity so temperature is set to 0.9

In [3]:
llm_answer = OpenAI(model="gpt-3.5-turbo", temperature=0)
llm_follow_up_question = OpenAI(model="gpt-3.5-turbo", temperature=0.9)
embed_model=OpenAIEmbedding()

### 2-Service context is created

In [4]:
service_context = ServiceContext.from_defaults(
            chunk_size=256,
            llm=llm_answer,
            embed_model=embed_model
)
service_context_follow_up = ServiceContext.from_defaults(
            chunk_size=256,
            llm=llm_answer,
            embed_model=embed_model
)
set_global_service_context(service_context)

/var/folders/bc/k6hr84f57gqbdw0648c3tblw0000gn/T/ipykernel_31857/360715135.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
/var/folders/bc/k6hr84f57gqbdw0648c3tblw0000gn/T/ipykernel_31857/360715135.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_follow_up = ServiceContext.from_defaults(


### For chunking llmsherpa api is used
### It chunk based upo the layout.The main advantage using this you get similar information in a chunk

In [11]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "Yash-Anchaliya.pdf" 
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)


### Creating a vector-index

In [ ]:
index = VectorStoreIndex([])
for chunk in doc.chunks():
    index.insert(Document(text=chunk.to_context_text(), extra_info={}))

### Printing chunks which are generated

In [10]:
for chunk in doc.chunks():
    print(chunk.to_context_text())
    print("------------------------------")
    print("##############################")


YASH ANCHALIYA ACADEMIC PROFILE
------------------------------
##############################

 | Degree/Certificate | Institution | Percentage/CGPA | Year
 | --- | --- | --- | ---
 | IDD | Bio-Medical Engineering IIT (BHU), Varanasi | 8.30 | 2024
 | CBSE (XII) | GURU SIDDHARTH INT SCHOOL BEAWAR | 88.200 | 2018
 | CBSE (X) | ST PAUL'S SR SEC SCHOOL BEAWAR | 91.200 | 2016

------------------------------
##############################
SKILLS
Language - C++, Python.
Technologies - Numpy, Pandas, OpenCV, Sklearn, Matplotlib, Pytorch, Keras, Tensorflow, Git.
Interest - DSA, Machine Learning, Deep Learning, Computer Vision, Generative AI, NLP, Statistics,OOPS.
------------------------------
##############################
INTERNSHIP/TRAINING > May 2023 - July 2023Data Science Intern | Walmart
Automated Creative Generation using Generative AI Worked in Generative AI Advertising Tech team to build automated creatives for advertisers using Stable Diffusion.
------------------------------
######

### Generating chat engine for question-answering and query engine for follow-up question

In [12]:
chat_engine = index.as_chat_engine()
query_engine  = index.as_query_engine(service_context= service_context_follow_up)

### generating results based on prompts

In [15]:
prompt ="tell me about yash anchaliya achivements"

In [13]:
answer = chat_engine.chat(prompt)

## Answer for the prompt

In [14]:
print(answer)

Yash Anchaliya has achieved the following:
1. Selected among the top 2% out of over 4.5 lakh candidates in Embark'21 conducted by Upraised.
2. Solved over 1000 problems on platforms like Leetcode and Codeforces.
3. Finalist in a Data Science Hackathon organized by Kharagpur Data Analytics Group at IIT Kharagpur.


### Follow-up question using llama-index query engine

In [16]:
follow_up_question = query_engine.query(f"generate followup question based upom {prompt}")

### Follow-up Question

In [17]:
print(follow_up_question)

What kind of competitions or challenges has Yash Anchaliya participated in to showcase their skills and abilities?
